In [59]:
!pip install liblas

In [13]:
import laspy

las = laspy.read('../laz/iowa.laz')
# with laspy.open('../laz/iowa.laz') as lidarfile:
# #     print("Points from header", lidarfile.header.point_count)
#     lazfile=lidarfile.read()
#     print(lazfile)
    
    

LaspyException: No LazBackend selected, cannot decompress data

In [19]:
!pip install pylas[lazrs,laszip]

     |████████████████████████████████| 1.2 MB 27.7 MB/s eta 0:00:01
  Created wheel for pylas: filename=pylas-0.4.2-py3-none-any.whl size=40342 sha256=49dd5af2a0a4aecc52900d7ca2126cb44848fcb1edf08aca4ac2741b24c07e7e
  Stored in directory: /home/blaise_papa/.cache/pip/wheels/59/55/b7/2b2a5728e028d77f697151a925f5cfa65c15e8fa1a9d92abdd
Successfully built pylas
  Attempting uninstall: pylas
    Found existing installation: pylas 0.4.3
    Uninstalling pylas-0.4.3:
      Successfully uninstalled pylas-0.4.3


In [61]:
import pylas 

las=pylas.read('../laz/iowa.laz')

In [62]:
las=pylas.convert(las)
las.write('../las/iowa.las')

In [63]:
import laspy

lasfile=laspy.read('../las/iowa.las')

In [64]:
lasfile.x

<ScaledArrayView([-10423180.01 -10423175.18 -10423172.82 ... -10424368.48 -10424375.46
 -10424380.77])>

In [65]:
with laspy.open('../las/iowa.las') as f:
    print(f"Point format:       {f.header.point_format}")
    print(f"Number of points:   {f.header.point_count}")
    print(f"Number of vlrs:     {len(f.header.vlrs)}")

Point format:       <PointFormat(3, 0 bytes of extra dims)>
Number of points:   1436158
Number of vlrs:     2


In [66]:
import numpy as np
lasfile_points=np.array((lasfile.x,lasfile.y,lasfile.z,lasfile.intensity,lasfile.raw_classification,lasfile.scan_angle_rank)).transpose()
lasfile_points

array([[-1.04231800e+07,  5.16577762e+06,  2.72700000e+02,
         6.80000000e+01,  2.00000000e+00,  0.00000000e+00],
       [-1.04231752e+07,  5.16577853e+06,  2.72800000e+02,
         4.20000000e+01,  2.00000000e+00,  1.00000000e+00],
       [-1.04231728e+07,  5.16577901e+06,  2.73100000e+02,
         5.80000000e+01,  2.00000000e+00,  1.00000000e+00],
       ...,
       [-1.04243685e+07,  5.16449968e+06,  2.85530000e+02,
         8.90000000e+01,  2.00000000e+00, -8.00000000e+00],
       [-1.04243755e+07,  5.16449907e+06,  2.85400000e+02,
         7.10000000e+01,  2.00000000e+00, -8.00000000e+00],
       [-1.04243808e+07,  5.16449899e+06,  2.85140000e+02,
         1.03000000e+02,  2.00000000e+00, -8.00000000e+00]])

In [67]:
#convert it to a dataframe
import pandas as pd

lasfile_df=pd.DataFrame(lasfile_points)
lasfile_df.columns=['lat','lon','elevation','intensity','classification','scan_angle']
lasfile_df

,lat,lon,elevation,intensity,classification,scan_angle
0,-10423180.01,5165777.62,272.70,68.0,2.0,0.0
1,-10423175.18,5165778.53,272.80,42.0,2.0,1.0
2,-10423172.82,5165779.01,273.10,58.0,2.0,1.0
3,-10423177.63,5165779.27,272.70,48.0,2.0,0.0
4,-10423180.02,5165778.81,272.54,71.0,2.0,0.0
...,...,...,...,...,...,...
1436153,-10424341.58,5164498.82,285.15,36.0,2.0,-9.0
1436154,-10424313.93,5164494.79,283.54,9.0,2.0,-10.0
1436155,-10424368.48,5164499.68,285.53,89.0,2.0,-8.0
1436156,-10424375.46,5164499.07,285.40,71.0,2.0,-8.0


In [68]:
##plot the 3d render
import numpy as np
import plotly.offline as go_offline
import plotly.graph_objects as go

In [69]:
def distance(x1,y1,x2,y2):
    d=np.sqrt((x1-x2)**2+(y1-y2)**2)
    return d



In [70]:
def idw_npoint(xz,yz,n_point,p):
    r=10 #block radius iteration distance
    nf=0
    while nf<=n_point: #will stop when np reaching at least n_point
        x_block=[]
        y_block=[]
        z_block=[]
        r +=10 # add 10 unit each iteration
        xr_min=xz-r
        xr_max=xz+r
        yr_min=yz-r
        yr_max=yz+r
        for i in range(len(x)):
            # condition to test if a point is within the block
            if ((x[i]>=xr_min and x[i]<=xr_max) and (y[i]>=yr_min and y[i]<=yr_max)):
                x_block.append(x[i])
                y_block.append(y[i])
                z_block.append(z[i])
        nf=len(x_block) #calculate number of point in the block
    
    #calculate weight based on distance and p value
    w_list=[]
    for j in range(len(x_block)):
        d=distance(xz,yz,x_block[j],y_block[j])
        if d>0:
            w=1/(d**p)
            w_list.append(w)
            z0=0
        else:
            w_list.append(0) #if meet this condition, it means d<=0, weight is set to 0
    
    #check if there is 0 in weight list
    w_check=0 in w_list
    if w_check==True:
        idx=w_list.index(0) # find index for weight=0
        z_idw=z_block[idx] # set the value to the current sample value
    else:
        wt=np.transpose(w_list)
        z_idw=np.dot(z_block,wt)/sum(w_list) # idw calculation using dot product
    return z_idw

In [71]:
x=lasfile_df['lat'][:3900]
y=lasfile_df['lon'][:3900]
z=lasfile_df['elevation'][:3900]



In [72]:
n=100 #number of interpolation point for x and y axis
x_min=min(x)
x_max=max(x)
y_min=min(y)
y_max=max(y)
w=x_max-x_min #width
h=y_max-y_min #length
wn=w/n #x interval
hn=h/n #y interval

#list to store interpolation point and elevation
y_init=y_min
x_init=x_min
x_idw_list=[]
y_idw_list=[]
z_head=[]
for i in range(n):
    print("Iteration {}".format(i))
    xz=x_init+wn*i
    yz=y_init+hn*i
    y_idw_list.append(yz)
    x_idw_list.append(xz)
    z_idw_list=[]
    for j in range(n):
        xz=x_init+wn*j
        z_idw=idw_npoint(xz,yz,5,1.5) #min. point=5, p=1.5
        z_idw_list.append(z_idw)
    z_head.append(z_idw_list)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [73]:
fig=go.Figure()
fig.add_trace(go.Surface(z=z_head,x=x_idw_list,y=y_idw_list,colorscale=[[0, "rgb(166,206,227)"],
                [0.25, "rgb(31,120,180)"],
                [0.45, "rgb(178,223,138)"],
                [0.65, "rgb(51,160,44)"],
                [0.85, "rgb(251,154,153)"],
                [1, "rgb(227,26,28)"]]))
fig.update_layout(scene=dict(aspectratio=dict(x=2, y=2, z=0.5),xaxis = dict(range=[x_min,x_max],),yaxis = dict(range=[y_min,y_max])))
go_offline.plot(fig,filename='../renders/iowa.html',validate=True, auto_open=False)

'../renders/iowa.html'